# Part 1 - Combine Sp22 Course Info with Requirements #

## 1. Read the 2022 course schedule into a DataFrame ##

In [2]:
from bs4 import BeautifulSoup
import re
import pandas as pd

with open('S22CSSchedule.html', 'r') as f:

    contents = f.read()

    soup = BeautifulSoup(contents, "html.parser")
    
#Extracting Number-Section
#<li class="row" id="csci-ga1144-002">

courses = []
names = []
instructors = []
times = []

class_li = soup.find_all('li', {'class': 'row'})
for course in class_li :
    
    #Extracting Number-Section
    courses.append(course.span.text.replace('\u200b', '').strip().split('\n')[0])
    
    #Extracting course name 
    individual_name = course.find('a', {'class': 'expand'}).get_text().strip().replace("\n", "")
    clean_name = re.sub(' +', ' ', individual_name)
    names.append(clean_name)
    
    #Extracting instructor name and time
    other_info = course.find_all("span", {'class': 'col-xs-12 col-sm-2'})
    individual_instructors = other_info[0].get_text().strip().replace("\n", "")
    clean_instructors = re.sub(' +', ' ', individual_instructors)
    instructors.append(clean_instructors)
    individual_time = other_info[1].get_text().strip().replace("\n", "")
    times.append(individual_time)
    
#FIGUREEE OUT HOW TO DEAL WITH MULTIPLE INSTRUCTORSSSSS (Don't just remove one)
#Add comma between the names
#Keeping main name of courses with two names

dataframe = pd.DataFrame()
dataframe["Number-Section"] = courses
dataframe["Name"] = names
dataframe["Instructor"] = instructors
dataframe["Time"] = times

#Separating number and section using regular expressions
separated_num_sec = pd.Series(courses).str.extract(r'(\w+.\w+.\d{4}).(\d{3})')
dataframe["Number"] = separated_num_sec[0]
dataframe["Section"] = separated_num_sec[1]

#Showing data frame info
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Number-Section  159 non-null    object
 1   Name            159 non-null    object
 2   Instructor      159 non-null    object
 3   Time            159 non-null    object
 4   Number          159 non-null    object
 5   Section         159 non-null    object
dtypes: object(6)
memory usage: 7.6+ KB


Reading Number-Section Column:
1. Dealing with multiple course numbers: whenever there were multiple course names, then the first name one was kept because first, the first name is consistent with the format/names of the other courses i.e. starts with CSCI instead of DS, and secondly, the first name has a hyperlink to a course page which suggests that it may be the more commonly used name.
2. Dealing with invisible space character: used the .replace() on course.span.text to remove the white space characters from courses number and section.

Reading Insturctor Column:

The names of all the instructors were kept as it feels wrong to just remove some instructors. There is no basis to remove an instructor as there is no categorization of primary, secondary instructors.

In [194]:
#Showing first 5 rows
dataframe.head(5)

,Number-Section,Name,Instructor,Time,Number,Section
0,CSCI-GA.1144-001,PAC II*,Hasan Aljabbouli,T 6:00-8:30PM,CSCI-GA.1144,001
1,CSCI-GA.1144-002,PAC II Recitation,Anish Mahishi,R 7:10-8:00PM,CSCI-GA.1144,002
2,CSCI-GA.1170-001,Fundamental Algorithms,Chee Yap,T 7:10-9:10PM,CSCI-GA.1170,001
3,CSCI-GA.1170-002,Fundamental Algorithms Recitation,Eli Goldin,R 8:10-9:00PM,CSCI-GA.1170,002
4,CSCI-GA.1170-003,Fundamental Algorithms Recitation,Zihan Feng,R 8:10-9:00PM,CSCI-GA.1170,003


In [195]:
#Showing last 5 rows
dataframe.tail(5)

,Number-Section,Name,Instructor,Time,Number,Section
154,CSCI-UA.0480-051,Special Topics: Parallel Computing,Mohamed Zahran,TR 2:00-3:15PM,CSCI-UA.0480,051
155,CSCI-UA.0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 9:30-10:45AM,CSCI-UA.0480,057
156,CSCI-UA.0480-061,Special Topics: Open Source Software Development,Joanna Klukowska,TR 12:30-1:45PM,CSCI-UA.0480,061
157,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 12:30-1:45PM,CSCI-UA.0480,069
158,CSCI-UA.0490-001,Special Topics: Programming Languages,Edward Yang,MW 4:55-6:10PM,CSCI-UA.0490,001


In [196]:
#Randomly sampling
dataframe.sample(n=5)

,Number-Section,Name,Instructor,Time,Number,Section
25,CSCI-GA.2820-001,DevOps and Agile Methodologies,John Rofrano,T 4:55-6:55PM,CSCI-GA.2820,001
41,CSCI-GA.3033-077,Special Topics: Big Data and ML Systems,Aurojit Panda,W 7:10-9:10PM,CSCI-GA.3033,077
42,CSCI-GA.3033-079,Special Topics: Mathematics of Deep Learning,Joan Bruna,R 4:55-6:55PM,CSCI-GA.3033,079
75,CSCI-UA.0002-006,Intro To Computer Programming (No Prior Experi...,Julie Lizardo,TR 11:00-12:15PM,CSCI-UA.0002,006
150,CSCI-UA.0478-001,Introduction To Cryptography,Marshall Ball,MW 9:30-10:45AM,CSCI-UA.0478,001


## 2. Read the 2022 course catalog into a DataFrame ##

In [197]:
with open('CourseCatalog.html', 'r') as file:

    contents2 = file.read()

    soup2 = BeautifulSoup(contents2, "html.parser")
    
#the frame should have the following columns:
#Number: the course number
#Prereqs: a text description of the prerequisites
#Points: the number of credits

course_numbers = []
prereqs = []
credits = []
 
course_li = soup2.find_all('li', {'class': 'col-sm-12'})

for course in course_li:
    
    p_list = course.select('p')
    
    #Extracting course number
    text = p_list[0].text.strip()
    course_number = re.search('\w+-\w+.\d{4}', text) #Cleaning up large text
    course_numbers.append(course_number.group(0))

    #Extracting Credits
    cred_text = p_list[1].text.strip().split('\n')[0].replace('Points.',"").strip()
    credits.append(cred_text)
    
    #Extracting Prereqs
    preq_text = p_list[2].text.replace("Prerequisites:", "").strip()
    if '\xa0' in preq_text:
        preq_text = preq_text.replace(u'\xa0', u' ') #Cleaning up
    prereqs.append(preq_text)
    
#credits.index(None)
#course_numbers[51]

#MAINT-GA.4747 has no credits data
#1-12 points
#course_li[56] #.select('p') #[1].text.strip().split('\n')[0].replace('Points.',"").strip()
#course_li[56].select('p')[2].text.replace("Prerequisites:", "").strip()

df2 = pd.DataFrame()
df2['Number'] = course_numbers
df2['Prereqs'] = prereqs
df2['Points'] = credits

In [198]:
#Showing dataframe info
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number   99 non-null     object
 1   Prereqs  99 non-null     object
 2   Points   99 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [199]:
#Showing first 5 rows
df2.head(5)

,Number,Prereqs,Points
0,CSCI-GA.1133,None,4
1,CSCI-GA.1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA.1170,At least one year of experience with a high-le...,3
3,CSCI-GA.1180,None,3
4,CSCI-GA.2110,Students taking this class should already have...,3


In [200]:
#Showing last 5 rows
df2.tail(5)

,Number,Prereqs,Points
94,CSCI-UA.0897,Restricted to declared computer science majors...,1 - 4
95,CSCI-UA.0898,Restricted to declared computer science majors...,1 - 4
96,CSCI-UA.0997,Permission of the department. Does not satisfy...,1 - 4
97,CSCI-UA.0998,Permission of the department. Does not satisfy...,1 - 4
98,FRSEM-UA.0597,"Some programming experience in Python, Java, J...",4


In [201]:
#Showing random 5 rows
df2.sample(n=5)

,Number,Prereqs,Points
26,CSCI-GA.2566,CSCI-GA 1180.,3
93,CSCI-UA.0521,Permission of the department.,4
64,CSCI-UA.0003,"Limited programming experience, or any score o...",4
75,CSCI-UA.0381,Data Science for Everyone (DS-UA 111) or equiv...,4
17,CSCI-GA.2421,Corequisite: linear algebra.,3


## 3. Put together both DataFrames ##

In [218]:
df3 = dataframe.merge(df2, how ='left', on='Number')
#del df3['Number']
#Moving number to start
df3 = df3[ ['Number'] + [ col for col in df3.columns if col != 'Number' ] ]
del df3['Number-Section']
del df3['Section']
df3.sample(n=5)

,Number,Name,Instructor,Time,Prereqs,Points
31,CSCI-GA.2945,Advanced Topics In Numerical Analysis: High Pe...,Benjamin Peherstorfer,M 1:25-3:15PM,Topics determine prerequisites.,3
75,CSCI-UA.0002,Intro To Computer Programming (No Prior Experi...,Julie Lizardo,TR 11:00-12:15PM,Three years of high school mathematics or equi...,4
22,CSCI-GA.2572,Deep Learning Lab,Jiachen Zhu,R 4:55-5:45PM,DS-GA 1001 Intro to Data Science or a graduate...,3
105,CSCI-UA.0102,Data Structures,Anasse Bari,TR 2:00-3:15PM,Introduction to Computer Science (CSCI-UA 101)...,4
71,CSCI-UA.0002,Intro To Computer Programming (No Prior Experi...,Susan Liao,TR 8:00-9:15AM,Three years of high school mathematics or equi...,4


## 4. Conclusion ##

1. Did you spot any anomalies, discrepancies, or unexpected data or relationships between data? If so, describe any problem(s) you saw. Additionally, describe how you might fix them (or if you already fixed them!)

There were discrepancies in the points data. Most were single digit, e.g. 4, some were a range, e.g. 1-4, and some were a combination of ranges categorized based on MS and PHD e.g. 1-3 (MS), 1-12 (PHD). This is a discrepancy as the data format is not consistent, additionally some of the ranges have extra spaces before and after the '-' i.e. 1 - 4 instead of 1-4. One way to fix this could be by dividing the points column into min and max credits. The lower end of the range would be the min and upper limit would be the max. If there is no range, then min == max. To fix the MS and PhD combined credits, we could add another column which specifies the level of education (undergrad, MS, PhD). e.g. If a course can be taken by both MS and PhD students, then we can duplicate the row. In the first row the column specifying the level would be MS and in the next row, the same would be PhD. Then for the respective rows, we could fill the min and max credits accordingly.

2. Lastly, based on the resulting DataFrame, describe the behavior of how=left on these particular DataFrames.

Setting how to left keeps every row of the first dataframe (dataframe). It looks for these rows based on the 'on' variable in the right dataframe (df2). If there are missing values of the “on” variable in the right dataframe (df2), it sets the result to NaN.